In [ ]:
import itertools
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
import pandas as pd
import config
import os
import json

In [ ]:
STAGE = 'ss'
MOD_PREFIX = "mod_smallimg3"
NEPOCH = 'latest'


DATAPATH = config.OUTPUT_PATH
base_dir = config.RAW_DATA_PATH
img_path = config.SCHULTHESS_DATAPATH
proc_dir = config.PROC_DATA_PATH

# #for rawq:
feature = 'rawq'
methods = 'comb_modalities'
folder = "2026-01-17_hdbscan"
run = "run28"  

# folder = "2025-11-19_hdbscan"
# methods = 'pipeline'
# run = "run10"  

anomalyscore_metric = "centre_mean"
cluster_col = "cluster_label"

folder_date = folder.split('_')[0]

if feature == 'rawq':
    filepath = os.path.join(proc_dir, folder, methods, run)
#     hdbscan_df = pd.read_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled.csv'))
    hdbscan_df = pd.read_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled.csv'))
    if methods == 'comb_modalities':
        hdbscan_df_test = pd.read_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled_test_results.csv'))
elif feature == 'img_features' or feature == 'img_raw':
        filepath = os.path.join(proc_dir, "radiographic_features", folder, run)
        hdbscan_df = pd.read_csv(os.path.join(filepath, f'{folder}_{run}_umap_hdbscan_scaled.csv'))
elif feature == 'agg':
      filepath = os.path.join(proc_dir, folder, run)
      hdbscan_df = pd.read_csv(os.path.join(filepath, f'{folder}_{run}_umap_hdbscan_scaled.csv'))
        
kl = pd.read_csv(os.path.join(base_dir,  "brul_knee_annotations.csv"))
kl2 = pd.read_csv(os.path.join(base_dir, "rosand1_knee_annotations.csv"))
mri = pd.read_csv(os.path.join(base_dir, '2025-09-25_mrismall.csv'))



In [ ]:
outputs = os.path.join(DATAPATH, 'outputs', 'dfs', STAGE)

filepath2 =  []
for file in os.listdir(outputs):
    if MOD_PREFIX in file and str(NEPOCH) in file and '_all' in file:
        filepath2.append(os.path.join(outputs, file))
dfs = []
for path in filepath2:
    df = pd.read_csv(path)[['id', anomalyscore_metric]]  # only keep id + target col
    dfs.append(df.rename(columns={anomalyscore_metric: os.path.basename(path)})) 
combined = dfs[0]
for df in dfs[1:]:
    combined = pd.merge(combined, df, on='id', how="inner")  # 'inner' keeps only common IDs

experiment_cols = [c for c in combined.columns if c != 'id']
combined["mean"] = combined[experiment_cols].mean(axis=1)
combined["std"] = combined[experiment_cols].std(axis=1)
combined.to_csv(os.path.join(outputs, f"{MOD_PREFIX}_{STAGE}_aggregated_scores.csv"), index = False)
combined['filepath'] = combined['id']
combined['id'] = combined['id'].apply(lambda x: x.split('/')[-1].replace('.png', ''))

In [ ]:
folder_df = "2025-09-11_data_exploration"
df_filename = "inmodi_data_questionnaire_kl_woSC.csv"
qdf = pd.read_csv(os.path.join(proc_dir, folder_df, df_filename))

In [ ]:
if 'KL-Score' not in hdbscan_df.columns:
    hdbscan_df = hdbscan_df.merge(qdf[['pain', 'age', 'ce_height',
       'ce_weight', 'ce_bmi', 'ce_fm', 'gender', 'OKS_score', 'UCLA_score',
       'FJS_score', 'KOOS_pain', 'KOOS_symptoms', 'KOOS_sport', 'KOOS_adl',
       'KOOS_qol', 'name', 'KL-Score', 'oks_q1', 'oks_q2', 'oks_q3', 'oks_q4',
       'oks_q5', 'oks_q6', 'oks_q7', 'oks_q8', 'oks_q9', 'oks_q10', 'oks_q11',
       'oks_q12', 'koos_s1', 'koos_s2', 'koos_s3', 'koos_s4', 'koos_s5',
       'koos_s6', 'koos_s7', 'koos_p1', 'koos_p2', 'koos_p3', 'koos_p4',
       'koos_p5', 'koos_p6', 'koos_p7', 'koos_p8', 'koos_p9', 'koos_a1',
       'koos_a2', 'koos_a3', 'koos_a4', 'koos_a5', 'koos_a6', 'koos_a7',
       'koos_a8', 'koos_a9', 'koos_a10', 'koos_a11', 'koos_a12', 'koos_a13',
       'koos_a14', 'koos_a15', 'koos_a16', 'koos_a17', 'koos_sp1', 'koos_sp2',
       'koos_sp3', 'koos_sp4', 'koos_sp5', 'koos_q1', 'koos_q2', 'koos_q3',
       'koos_q4']], left_on='id', right_on='name', how='left')
    hdbscan_df = hdbscan_df.merge(combined, on='id', how='left')

    try:
        hdbscan_df_test = hdbscan_df_test.merge(qdf[['pain', 'age', 'ce_height',
       'ce_weight', 'ce_bmi', 'ce_fm', 'gender', 'OKS_score', 'UCLA_score',
       'FJS_score', 'KOOS_pain', 'KOOS_symptoms', 'KOOS_sport', 'KOOS_adl',
       'KOOS_qol', 'name', 'KL-Score', 'oks_q1', 'oks_q2', 'oks_q3', 'oks_q4',
       'oks_q5', 'oks_q6', 'oks_q7', 'oks_q8', 'oks_q9', 'oks_q10', 'oks_q11',
       'oks_q12', 'koos_s1', 'koos_s2', 'koos_s3', 'koos_s4', 'koos_s5',
       'koos_s6', 'koos_s7', 'koos_p1', 'koos_p2', 'koos_p3', 'koos_p4',
       'koos_p5', 'koos_p6', 'koos_p7', 'koos_p8', 'koos_p9', 'koos_a1',
       'koos_a2', 'koos_a3', 'koos_a4', 'koos_a5', 'koos_a6', 'koos_a7',
       'koos_a8', 'koos_a9', 'koos_a10', 'koos_a11', 'koos_a12', 'koos_a13',
       'koos_a14', 'koos_a15', 'koos_a16', 'koos_a17', 'koos_sp1', 'koos_sp2',
       'koos_sp3', 'koos_sp4', 'koos_sp5', 'koos_q1', 'koos_q2', 'koos_q3',
       'koos_q4']], left_on='id', right_on='name', how='left')
        hdbscan_df_test = hdbscan_df_test.merge(combined, on='id', how='left')
    except:
        pass

In [ ]:
hdbscan_df_test['cluster_label'].unique()

In [ ]:
hdbscan_df['cluster_label'].unique()

In [ ]:
ques = pd.read_csv(os.path.join(base_dir, 'questionnaires_raw.csv')) #?

In [ ]:
try:
    hdbscan_df.drop(columns=['name_y', 'KL-Score_y'], inplace=True)
    hdbscan_df.rename(columns={'name_x': 'name', 'KL-Score_x': 'KL-Score'}, inplace=True)
except:
    pass

In [ ]:
print(len(hdbscan_df))
print(len(hdbscan_df_test))

In [ ]:
hdbscan_df.head()

In [ ]:
print(len(hdbscan_df))
print(len(hdbscan_df_test))

# Load Embeddings

In [ ]:
import joblib

In [ ]:
with open(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled_model_info.json')) as f:
    model_info= json.load(f)
if methods == 'comb_modalities':
    umap_info = model_info['params']['umap']
    nneigh = umap_info['n_neighbors']
    min_dist = umap_info['min_dist']
    metric = umap_info['metric']

    umap_folder = f'nneigh{nneigh}_mindist{min_dist}_metric{metric}'
    umap_path = os.path.join(proc_dir, '2026-01-16_umap_scaler_values', 'comb_modalities', umap_folder)

    with open(os.path.join(umap_path, 'smote_oversampled_data_artifacts.json')) as f:
        umap_artifacts = json.load(f)

    ids = umap_artifacts['ids']

    embeddings_path = os.path.join(umap_path, 'X_umap_embeddings.npy')
    embeddings = np.load(embeddings_path)

    embeddings_path_test = os.path.join(filepath, 'X_test_umap_embeddings.npy')
    embeddings_test = np.load(embeddings_path_test)

    embeddings_smote_path = os.path.join(umap_path, 'X_umap_samp_embeddings.npy')

    try:
        smote_path = os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_generated_samples_predicted.csv')
        smote_df = pd.read_csv(smote_path)
        #test if cluster_label in smote_df
        if 'cluster_label' not in smote_df.columns:
            raise FileNotFoundError
    except:
        pass

    smote_path = os.path.join(umap_path, 'SMOTE_generated_samples.csv')
    smote_df = pd.read_csv(smote_path)
    smote_embeddings = np.load(embeddings_smote_path)

    clusterer = joblib.load(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled_clusterer.pkl'))
    smote_df['cluster_label'] = clusterer.labels_

else:
    embeddings_path = os.path.join(filepath, "X_umap_embeddings.npy")
    embeddings = np.load(embeddings_path)

    embeddings_smote_path = os.path.join(filepath, 'X_umap_samp_embeddings.npy')
    smote_embeddings = np.load(embeddings_smote_path)

    smote_path = os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_generated_samples_predicted.csv')

    smote_df = pd.read_csv(smote_path)

    ids = model_info['files']['ids']

In [ ]:
print(len(hdbscan_df))
print(len(embeddings))

print(len(smote_df))
print(len(smote_embeddings))

In [ ]:
print(len(hdbscan_df_test))
print(len(embeddings_test))

In [ ]:
# hdbscan_df_test = hdbscan_df_test[hdbscan_df_test['cluster_label']!=-1]
ids_test = hdbscan_df_test['id'].tolist()

In [ ]:
print(len(hdbscan_df))
print(len(embeddings))

In [ ]:
hdbscan_df_wo = hdbscan_df[hdbscan_df['cluster_label']!=-1]
hdbscan_df_test_wo = hdbscan_df_test[hdbscan_df_test['cluster_label']!=-1]

In [ ]:
# ids = model_info['files']['ids']

if len(embeddings)>len(hdbscan_df_wo):
    #create mapping

    embeddings_won = embeddings[ids.index(hdbscan_df_wo['id'].iloc[0]): ids.index(hdbscan_df_wo['id'].iloc[-1])+1]
    embeddings_won = embeddings_won[[ids.index(i) for i in hdbscan_df_wo['id']]]
    ids_wo = [i for i in ids if i in hdbscan_df_wo['id'].tolist()]

if len(embeddings_test)>len(hdbscan_df_test_wo):
    embeddings_test_won = embeddings_test[ids_test.index(hdbscan_df_test_wo['id'].iloc[0]): ids_test.index(hdbscan_df_test_wo['id'].iloc[-1])+1]
    embeddings_test_won = embeddings_test_won[[ids_test.index(i) for i in hdbscan_df_test_wo['id']]]
    ids_test_wo = [i for i in ids_test if i in hdbscan_df_test_wo['id'].tolist()]

In [ ]:
print(len(hdbscan_df_wo))
print(len(embeddings_won))

In [ ]:
print(len(ids_test))
print(len(embeddings_test))

In [ ]:
clusters = hdbscan_df['cluster_label'].unique().tolist()
clusters.sort()

healthy_cluster = 0
healthy_centroid = embeddings[hdbscan_df['cluster_label'] == healthy_cluster].mean(axis=0)
healthy_spread = np.mean(np.linalg.norm(embeddings[hdbscan_df['cluster_label'] == healthy_cluster]-healthy_centroid, axis=1))

cluster_centroids = {}

for cluster in clusters:
    cluster_embeddings = embeddings[hdbscan_df['cluster_label'] == cluster]
    cluster_centroid = cluster_embeddings.mean(axis=0)
    spread = np.mean(np.linalg.norm(cluster_embeddings-cluster_centroid, axis=1))
    cluster_centroids[cluster] = {
        'centroid': cluster_centroid,
        'distance': np.linalg.norm(cluster_centroid-healthy_centroid),
        'spread': spread,
        'norm_distance': np.linalg.norm(cluster_centroid - healthy_centroid)/(healthy_spread+spread)
    }

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
healthy_mask = hdbscan_df['cluster_label'] == healthy_cluster
healthy_embeddings = embeddings[healthy_mask]
healthy_centroid = healthy_embeddings.mean(axis=0)

healthy_spread = np.mean(
    np.linalg.norm(
        healthy_embeddings - healthy_centroid,
        axis=1
    )
)
dist_to_healthy = cdist(embeddings, healthy_embeddings)

point_severity = dist_to_healthy.min(axis=1)

cluster_centroids = {}

for cluster in clusters:
    cluster_mask = hdbscan_df['cluster_label'] == cluster
    cluster_embeddings = embeddings[cluster_mask]

    # centroid (for reference / plotting)
    cluster_centroid = cluster_embeddings.mean(axis=0)

    # spread = mean distance to centroid
    spread = np.mean(
        np.linalg.norm(cluster_embeddings - cluster_centroid, axis=1)
    )

    # NEW: cluster distance = mean nearest-healthy distance
    cluster_distance = point_severity[cluster_mask].mean()
    print(f"Cluster {cluster}: distance to healthy = {cluster_distance}, spread = {spread}")

    # normalized distance
    denom = healthy_spread + spread
    norm_distance = cluster_distance / denom if denom > 0 else np.nan

    cluster_centroids[cluster] = {
        'centroid': cluster_centroid,
        'distance': cluster_distance,
        'spread': spread,
        'norm_distance': norm_distance
    }


In [ ]:
def normalized_distance(row, embeddings):
    c = row['cluster_label']
    point = embeddings[row.name]

    raw_distance = np.linalg.norm(point - healthy_centroid)
    norm = cluster_centroids[c]['spread'] + cluster_centroids[healthy_cluster]['spread']

    return raw_distance / norm if norm >0 else raw_distance #np.nan

## Order using SMOTE data

In [ ]:
healthy_mask = smote_df['cluster_label'] == healthy_cluster
healthy_embeddings = smote_embeddings[healthy_mask]
healthy_centroid = healthy_embeddings.mean(axis=0)

healthy_spread = np.mean(
    np.linalg.norm(
        healthy_embeddings - healthy_centroid,
        axis=1
    )
)
dist_to_healthy = cdist(smote_embeddings, healthy_embeddings)

point_severity = dist_to_healthy.min(axis=1)

cluster_centroids = {}

for cluster in clusters:
    cluster_mask = smote_df['cluster_label'] == cluster
    cluster_embeddings = smote_embeddings[cluster_mask]

    # centroid (for reference / plotting)
    cluster_centroid = cluster_embeddings.mean(axis=0)

    # spread = mean distance to centroid
    spread = np.mean(
        np.linalg.norm(cluster_embeddings - cluster_centroid, axis=1)
    )

    # NEW: cluster distance = mean nearest-healthy distance
    cluster_distance = point_severity[cluster_mask].mean()
    print(f"Cluster {cluster}: distance to healthy = {cluster_distance}, spread = {spread}")

    # normalized distance
    denom = healthy_spread + spread
    norm_distance = cluster_distance / denom if denom > 0 else np.nan

    cluster_centroids[cluster] = {
        'centroid': cluster_centroid,
        'distance': cluster_distance,
        'spread': spread,
        'norm_distance': norm_distance
    }


In [ ]:
for cluster in cluster_centroids:
    print(f"Cluster {cluster}: norm. distance is {cluster_centroids[cluster]['norm_distance']}, with spread {cluster_centroids[cluster]['spread']}")

In [ ]:
orderd = sorted(cluster_centroids.items(), key=lambda x: x[1]['norm_distance'])
order_wonoised = [item for item in orderd if item[0] != -1]
order = {cluster: rank for rank, (cluster, _) in enumerate(order_wonoised)}
cluster_mapping = {cluster: rank for rank, (cluster, _) in enumerate(order_wonoised)}

In [ ]:
def normalized_distance(row, embeddings):
    c = row['cluster_label']

    if pd.isna(c):
        return np.nan   # or raw_distance, depending on your intent

    point = embeddings[row.name]
    raw_distance = np.linalg.norm(point - healthy_centroid)

    norm = (
        cluster_centroids[c]['spread']
        + cluster_centroids[healthy_cluster]['spread']
    )

    return raw_distance / norm if norm > 0 else np.nan

# smote_df['old_label'] = smote_df['cluster_label']
# smote_df['cluster_label'] = smote_df['cluster_label'].map(cluster_mapping)
smote_df['severity_score'] = np.linalg.norm(smote_embeddings - healthy_centroid, axis=1)
smote_df['severity_score_norm'] = smote_df.apply(normalized_distance, axis = 1, embeddings= smote_embeddings)
# smote_df.to_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_smote_clusters.csv'), index=False)

In [ ]:
smote_df.groupby("cluster_label")["severity_score"].describe()

In [ ]:
smote_df.groupby("cluster_label")["severity_score_norm"].describe()

In [ ]:
smote_df['old_label'] = smote_df['cluster_label']
smote_df['cluster_label'] = smote_df['cluster_label'].map(cluster_mapping)
smote_df.to_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_smote_clusters.csv'), index=False)

### Get updated original dataset

In [ ]:
hdbscan_df['severity_score'] = np.linalg.norm(embeddings - healthy_centroid, axis=1)
hdbscan_df['severity_score_norm'] = hdbscan_df.apply(normalized_distance, axis = 1, embeddings= embeddings)
hdbscan_df['severity_score_norm_cluster'] = hdbscan_df['cluster_label'].apply(lambda x: cluster_centroids[x]['norm_distance'])
hdbscan_df['severity_score_manhatten'] = np.abs(np.linalg.norm(embeddings - healthy_centroid, ord=1, axis=1))
hdbscan_df['severity_score_chebyshev'] = np.abs(np.linalg.norm(embeddings - healthy_centroid, ord=np.inf, axis=1))
try:
    hdbscan_df_test['severity_score'] = np.linalg.norm(embeddings_test - healthy_centroid, axis=1)
    hdbscan_df_test['severity_score_norm'] = hdbscan_df_test.apply(normalized_distance, axis = 1, embeddings= embeddings_test)
    hdbscan_df_test['severity_score_norm_cluster'] = hdbscan_df_test['cluster_label'].apply(lambda x: cluster_centroids[x]['norm_distance'])
except:
    pass

In [ ]:
hdbscan_df_test['severity_score'] = np.linalg.norm(embeddings_test - healthy_centroid, axis=1)
hdbscan_df_test['severity_score_norm'] = hdbscan_df_test.apply(normalized_distance, axis = 1, embeddings= embeddings_test)
hdbscan_df_test['severity_score_norm_cluster'] = hdbscan_df_test['cluster_label'].apply(lambda x: cluster_centroids[x]['norm_distance'])

# Severity Score Behaviour for each cluster label

In [ ]:
hdbscan_df_wo['old_cluster'] = hdbscan_df_wo['cluster_label']
hdbscan_df_wo['cluster_label'] = hdbscan_df_wo['cluster_label'].map(cluster_mapping)
#replace cluster_label Nan with -1
hdbscan_df_wo['cluster_label'].fillna(-1, inplace=True)

In [ ]:
hdbscan_df_test_wo['old_cluster'] = hdbscan_df_test_wo['cluster_label']
hdbscan_df_test_wo['cluster_label'] = hdbscan_df_test_wo['cluster_label'].map(cluster_mapping)
#replace cluster_label Nan with -1
hdbscan_df_test_wo['cluster_label'].fillna(-1, inplace=True)

In [ ]:
cluster_centroids_new = {}
for old, new in cluster_mapping.items():
    cluster_centroids_new[new] = cluster_centroids[old]

hdbscan_df['old_cluster'] = hdbscan_df['cluster_label']
hdbscan_df['cluster_label'] = hdbscan_df['cluster_label'].map(cluster_mapping)
#replace cluster_label Nan with -1
hdbscan_df['cluster_label'].fillna(-1, inplace=True)

try:
    hdbscan_df_test['old_cluster'] = hdbscan_df_test['cluster_label']
    hdbscan_df_test['cluster_label'] = hdbscan_df_test['cluster_label'].map(cluster_mapping)
    #replace cluster_label Nan with -1
    hdbscan_df_test['cluster_label'].fillna(-1, inplace=True)
except:
    pass

import seaborn as sns
sns.violinplot(data=hdbscan_df, x="cluster_label", y="severity_score_norm")

In [ ]:
try:
    sns.violinplot(data=hdbscan_df_test, x="cluster_label", y="severity_score_norm")
except:
    pass


In [ ]:
try:
    hdbscan_df_test['train_test'] = 'test'
    hdbscan_df['train_test'] = 'train'

    df = pd.concat([hdbscan_df, hdbscan_df_test], ignore_index=True)
except:
    df = hdbscan_df
    

df.to_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_severity_scores.csv'), index=False)

# SMOTE

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#get colormap used here
import matplotlib.cm as cm
import matplotlib.colors as mcolors
cmap = cm.get_cmap('Set3', len(cluster_centroids_new)+1)

#for cluster colors include noise point -1 not in clster_centroids_new
cluster_colors = {-1: mcolors.to_hex(cmap(0))}
cluster_colors.update({cluster: mcolors.to_hex(cmap(i+1)) for i, cluster in enumerate(cluster_centroids_new.keys())})

In [ ]:
df['cluster_label'] = df['cluster_label'].astype(int)

In [ ]:
cluster_colors = {str(k): v for k, v in cluster_colors.items()}

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=df[df['cluster_label']!=-1],
    x="cluster_label",
    y="severity_score_norm",
    # palette=cluster_colors,
    color = '#80b1d3',
    width=0.6,
    showfliers=False,     # hide outliers
    linewidth=1.5         # thicker box borders
)


plt.title("Severity Score Distribution per Cluster", fontsize=16, pad=15)
plt.xlabel("Cluster Label", fontsize=13)
plt.ylabel("Severity Score", fontsize=13)
plt.grid(axis="y", alpha=0.25)
plt.tight_layout()
plt.show()



In [ ]:
cluster_colors = {int(k): v for k, v in cluster_colors.items()}

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(
    data=df[df['cluster_label']!=-1],
    x="KL-Score",
    y="severity_score_norm",
    hue="cluster_label",
    palette=cluster_colors,
    estimator="mean",
    errorbar="sd",   # or None if you want no error bars
    dodge=True
)

plt.legend(
    title="Cluster",
    bbox_to_anchor=(1.02, 1),
    loc="upper left",
    borderaxespad=0
)




plt.title("Severity Score Distribution per KL-Score", fontsize=16, pad=15)
plt.xlabel("KL-Score", fontsize=13)
plt.ylabel("Severity Score (normalized)", fontsize=13)
plt.grid(axis="y", alpha=0.25)
plt.tight_layout()
plt.show()


In [ ]:
cluster_mapping.update({-1: -1})

In [ ]:
cluster_colors_old = {int(k): v for k, v in cluster_colors.items()}

In [ ]:
#cluster_colors use colors for clusters using cluster_mapping in reverse
cluster_colors_old = {old: cluster_colors_old[new] for old, new in cluster_mapping.items()}

In [ ]:
#turn cluster_colors keys into str
cluster_colors_old = {str(k): v for k, v in cluster_colors_old.items()}

In [ ]:

plt.figure(figsize=(10, 6))
sns.boxplot(
    data=df,
    x="old_cluster",
    y="severity_score_norm",
    palette=cluster_colors_old, #why is this not working?
    width=0.6,
    showfliers=False,     # hide outliers
    linewidth=1.5         # thicker box borders
)

plt.title("Severity Score Distribution per Cluster", fontsize=16, pad=15)
plt.xlabel("Cluster Label", fontsize=13)
plt.ylabel("Severity Score", fontsize=13)
plt.grid(axis="y", alpha=0.25)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=df[df['cluster_label']!=-1],
    x="KL-Score",
    y="severity_score_norm",
    color = '#ffed6f',
    # palette=cluster_colors_old, #why is this not working?
    width=0.6,
    showfliers=False,     # hide outliers
    linewidth=1.5         # thicker box borders
)

plt.title("Severity Score Distribution per KL-Score", fontsize=16, pad=15)
plt.xlabel("KL-Score", fontsize=13)
plt.ylabel("Severity Score", fontsize=13)
plt.grid(axis="y", alpha=0.25)
plt.tight_layout()
plt.show()

## Anomaly Scores

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=df,
    x="cluster_label",
    y="mean",
    hue = 'cluster_label',
    palette=cluster_colors, #why is this not working?
    width=0.6,
    showfliers=False,     # hide outliers
    linewidth=1.5         # thicker box borders
)

plt.title("Anomaly Score Distribution per Cluster", fontsize=16, pad=15)
plt.xlabel("Cluster Label", fontsize=13)
plt.ylabel("Anomaly Score", fontsize=13)
plt.grid(axis="y", alpha=0.25)
plt.tight_layout()
plt.show()

# Plot original data & SMOTE in embedding space

In [ ]:
from utils.hdbscan_utils import plot_hdbscan, plot_hdbscan_wcentroid

## New Order

In [ ]:
if methods == 'comb_modalities':
    np.save(os.path.join(filepath, f'X_umap_embeddings_wonoise.npy'), embeddings_won)
    np.save(os.path.join(filepath, f'X_text_umap_embeddings_wonoise.npy'), embeddings_test_won)
    np.save(os.path.join(filepath, f'X_umap_samp_embeddings_wonoise.npy'), smote_embeddings)

In [ ]:
print(embeddings.shape)
print(len(hdbscan_df))

In [ ]:
plot_hdbscan_wcentroid(X = embeddings,
            labels = hdbscan_df['cluster_label'],
            centroids_dict=cluster_centroids_new,
            n_clusters=len(cluster_mapping))

In [ ]:
print(len(hdbscan_df_test))
print(len(embeddings_test))

In [ ]:
plot_hdbscan_wcentroid(X = embeddings_test,
            labels = hdbscan_df_test['cluster_label'],
            centroids_dict=cluster_centroids_new,
            n_clusters=len(cluster_mapping))

In [ ]:
full_embeddings = np.concatenate([embeddings_won, embeddings_test_won], axis=0)
print(full_embeddings.shape)

In [ ]:
plot_hdbscan_wcentroid(X = full_embeddings,
            labels = pd.concat([hdbscan_df_wo['cluster_label'], hdbscan_df_test_wo['cluster_label']]),
            centroids_dict=cluster_centroids_new,
            n_clusters=len(hdbscan_df_wo['cluster_label'].unique()))

In [ ]:
plot_hdbscan_wcentroid(X = smote_embeddings,
            labels = smote_df['cluster_label'],
            centroids_dict=cluster_centroids_new,
            n_clusters=len(cluster_mapping))

# Image Folder per Cluster

In [ ]:
# Get images per cluster
import shutil

output_path = os.path.join(DATAPATH, 'outputs', 'clusterimages', f'{methods}_{run}_umap_hdbscan_cluster_label')
os.makedirs(output_path, exist_ok=True)
img_path = os.path.join(base_dir, 'images_knee')
for cluster in df['cluster_label'].unique():
    ids = df[df['cluster_label'] == cluster]['id'].tolist()
    cluster_dir = os.path.join(output_path, f'cluster_{cluster}')
    os.makedirs(cluster_dir, exist_ok=True)
    for id_ in ids:
        img_file = os.path.join(img_path, f'{id_}.png')
        if os.path.exists(img_file):
            dest_file = os.path.join(cluster_dir, f'{id_}.png')
            if not os.path.exists(dest_file):
                shutil.copy2(img_file, dest_file)

# Get Images per Cluster

In [ ]:
import random
import matplotlib.pyplot as plt
from PIL import Image

def get_image_id(img_path):
    return os.path.splitext(os.path.basename(img_path))[0]


def show_cluster_examples(output_path, df, max_grid=3, seed=42):
    random.seed(seed)

    # Build KL lookup
    kl_lookup = dict(zip(df['name'], df['KL-Score']))

    clusters = sorted([c for c in os.listdir(output_path) if c.startswith("cluster_")])
    img_names_d = {}

    for cluster in clusters:
        cluster_dir = os.path.join(output_path, cluster)

        imgs = [
            os.path.join(cluster_dir, f)
            for f in os.listdir(cluster_dir)
            if f.lower().endswith((".png", ".jpg", ".jpeg"))
        ]

        if len(imgs) == 0:
            print(f"No images found in {cluster}")
            continue

        # -----------------------------
        # Group images by KL-score
        # -----------------------------
        imgs_by_kl = {}
        for img_path in imgs:
            img_id = get_image_id(img_path)
            kl = kl_lookup.get(img_id, None)
            if kl is None:
                continue
            imgs_by_kl.setdefault(kl, []).append(img_path)

        available_kls = sorted(imgs_by_kl.keys())

        if len(available_kls) == 0:
            print(f"No KL-labelled images in {cluster}")
            continue

        # -----------------------------
        # Ensure at least one per KL
        # -----------------------------
        sample_imgs = []

        for kl in available_kls:
            sample_imgs.append(random.choice(imgs_by_kl[kl]))

        # -----------------------------
        # Fill remaining slots (if any)
        # -----------------------------
        max_images = max_grid * max_grid
        remaining = max_images - len(sample_imgs)

        if remaining > 0:
            remaining_imgs = [
                img for img in imgs
                if img not in sample_imgs
            ]

            if len(remaining_imgs) > 0:
                sample_imgs.extend(
                    random.sample(
                        remaining_imgs,
                        min(remaining, len(remaining_imgs))
                    )
                )

        # img_names_l.append(sample_imgs)
        img_names_d[cluster] = imgs

        # -----------------------------
        # Plot
        # -----------------------------
        n_images = len(sample_imgs)
        cols = min(max_grid, n_images)
        rows = (n_images + cols - 1) // cols

        fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
        fig.suptitle(
            f"Cluster {cluster.replace('cluster_', '')}: Example Images",
            fontsize=18
        )

        if rows == 1 and cols == 1:
            axes = [axes]
        elif rows == 1:
            axes = axes
        else:
            axes = axes.flatten()

        for ax, img_path in zip(axes, sample_imgs):
            img = Image.open(img_path).convert("L")
            ax.imshow(img, cmap="gray")
            ax.axis("off")

            img_id = get_image_id(img_path)
            kl = kl_lookup.get(img_id, "KL ?")

            ax.text(
                0.02, 0.02,
                f"KL {kl}",
                transform=ax.transAxes,
                fontsize=14,
                color="white",
                ha="left",
                va="bottom",
                bbox=dict(facecolor="black", alpha=0.5, pad=2)
            )

        # Hide unused axes
        for ax in axes[len(sample_imgs):]:
            ax.axis("off")

        plt.tight_layout()
        plt.show()

    return img_names_d


In [ ]:
img_names_d = show_cluster_examples(output_path, df, max_grid=3, seed=8)


## Get images per cluster, grouped by KL score

In [ ]:
def get_image_id(img_path):
    return os.path.splitext(os.path.basename(img_path))[0]


def show_cluster_examples(output_path, df, max_grid=3, seed=42):
    random.seed(seed)

    # Build KL lookup
    kl_lookup = dict(zip(df['name'], df['KL-Score']))
    mri_cart_lookup = dict(zip(df['name'], df['mri_cart_yn']))
    mri_osteo_lookup = dict(zip(df['name'], df['mri_osteo_yn']))
    mri_bml_lookup = dict(zip(df['name'], df['mri_bml_yn']))

    clusters = sorted([c for c in os.listdir(output_path) if c.startswith("cluster_")])

    for cluster in clusters:
        cluster_dir = os.path.join(output_path, cluster)

        imgs = [
            os.path.join(cluster_dir, f)
            for f in os.listdir(cluster_dir)
            if f.lower().endswith((".png", ".jpg", ".jpeg"))
        ]

        if len(imgs) == 0:
            print(f"No images found in {cluster}")
            continue

        # -----------------------------
        # Group images by KL-score
        # -----------------------------
        imgs_by_kl = {}
        for img_path in imgs:
            img_id = get_image_id(img_path)
            kl = kl_lookup.get(img_id, None)
            if kl is None:
                continue
            mri_cart = mri_cart_lookup.get(img_id, None)
            mri_osteo = mri_osteo_lookup.get(img_id, None)
            mri_bml = mri_bml_lookup.get(img_id, None)
            # imgs_by_kl.setdefault(kl, []).append(img_path)
            imgs_by_kl.setdefault(kl, []).append({
                    "path": img_path,
                    "cart": mri_cart,
                    "osteo": mri_osteo,
                    "bml": mri_bml
                })


        available_kls = sorted(imgs_by_kl.keys())

        if len(available_kls) == 0:
            print(f"No KL-labelled images in {cluster}")
            continue

        # -----------------------------
        # Prepare grid: one row per KL
        # -----------------------------
        n_rows = len(available_kls)
        n_cols = max_grid

        fig, axes = plt.subplots(
            n_rows,
            n_cols,
            figsize=(4 * n_cols, 4 * n_rows),
            squeeze=False
        )

        fig.suptitle(
            f"Cluster {cluster.replace('cluster_', '')} - Example Images",
            fontsize=18
        )

        # -----------------------------
        # Fill grid
        # -----------------------------
        for row_idx, kl in enumerate(available_kls):
            kl_imgs = imgs_by_kl[kl]

            # sample up to max_grid images for this KL
            if len(kl_imgs) > max_grid:
                kl_imgs = random.sample(kl_imgs, max_grid)

            for col_idx in range(n_cols):
                ax = axes[row_idx, col_idx]

                if col_idx < len(kl_imgs):
                    item = kl_imgs[col_idx]
                    img = Image.open(item['path']).convert("L")
                    ax.imshow(img, cmap="gray")

                    ax.text(
                        0.02, 0.02,
                        f"KL {kl}",
                        transform=ax.transAxes,
                        fontsize=14,
                        color="white",
                        ha="left",
                        va="bottom",
                        bbox=dict(facecolor="black", alpha=0.5, pad=2)
                    )
                    ax.text(
                                0.98, 0.98,                 # top-right corner
                                  f"Cart: {item['cart']}\n"
                                    f"Osteo: {item['osteo']}\n"
                                    f"BML: {item['bml']}",        # any extra info you want
                                transform=ax.transAxes,
                                fontsize=14,
                                color="white",
                                ha="right",                 # horizontal alignment
                                va="top",                   # vertical alignment
                                bbox=dict(
                                    facecolor="black",
                                    alpha=0.5,
                                    pad=3
                                )
                            )

                else:
                    ax.axis("off")

                ax.set_xticks([])
                ax.set_yticks([])

        plt.tight_layout(rect=[0, 0, 1, 0.93])
        plt.show()


In [ ]:
df_mri = df.merge(mri, how = 'left', left_on='name', right_on = 'id')

In [ ]:
df_mri = df_mri.dropna(subset=['mri_side'])

In [ ]:
show_cluster_examples(output_path, df_mri,  max_grid = 3, seed = 8)

## Column KL-score, row clsuter

In [ ]:
clusters = sorted([c for c in os.listdir(output_path) if c.startswith("cluster_")])

In [ ]:
clusters.remove('cluster_-1')

In [ ]:
def get_image_id(img_path):
    return os.path.splitext(os.path.basename(img_path))[0]


def show_cluster_examples_klscores(output_path, df, seed=42):
    random.seed(seed)

    # Build KL lookup
    kl_lookup = dict(zip(df['name'], df['KL-Score']))

    clusters = sorted([c for c in os.listdir(output_path) if c.startswith("cluster_")])
    clusters.remove("cluster_-1")  # Exclude noise cluster

    # -------------------------------------------------
    # First pass: collect images per (cluster, KL)
    # -------------------------------------------------
    cluster_kl_images = {}
    all_kls = set()

    for cluster in clusters:
        cluster_dir = os.path.join(output_path, cluster)

        imgs = [
            os.path.join(cluster_dir, f)
            for f in os.listdir(cluster_dir)
            if f.lower().endswith((".png", ".jpg", ".jpeg"))
        ]

        kl_map = {}
        for img_path in imgs:
            img_id = get_image_id(img_path)
            kl = kl_lookup.get(img_id, None)
            if kl is None:
                continue
            kl_map.setdefault(kl, []).append(img_path)
            all_kls.add(kl)

        cluster_kl_images[cluster] = kl_map

    all_kls = sorted(all_kls)

    if len(all_kls) == 0:
        print("No KL-labelled images found.")
        return

    # -------------------------------------------------
    # Prepare grid: rows = clusters, cols = KLs
    # -------------------------------------------------
    n_rows = len(clusters)
    n_cols = len(all_kls)

    fig, axes = plt.subplots(
        n_rows,
        n_cols,
        figsize=(4 * n_cols, 4 * n_rows),
        squeeze=False
    )

    fig.suptitle(
        "Cluster × KL example images",
        fontsize=20
    )

    # -------------------------------------------------
    # Column titles (KL scores)
    # -------------------------------------------------
    for col_idx, kl in enumerate(all_kls):
        axes[0, col_idx].set_title(f"KL {kl}", fontsize=14)

    # -------------------------------------------------
    # Fill grid
    # -------------------------------------------------
    for row_idx, cluster in enumerate(clusters):
        kl_map = cluster_kl_images.get(cluster, {})

        # Row label (cluster)
        axes[row_idx, 0].set_ylabel(
            cluster.replace("cluster_", "Cluster "),
            fontsize=14,
            rotation=0,
            labelpad=60,
            va="center"
        )

        for col_idx, kl in enumerate(all_kls):
            ax = axes[row_idx, col_idx]

            imgs_for_cell = kl_map.get(kl, [])

            if len(imgs_for_cell) > 0:
                img_path = random.choice(imgs_for_cell)
                img = Image.open(img_path).convert("L")
                ax.imshow(img, cmap="gray")
                ax.set_xticks([])
                ax.set_yticks([])
            else:
                if col_idx != 0:   # keep first column axis alive
                    ax.axis("off")
                    # ax.set_visible(False)
                    # ax.remove()
                else:
                    ax.set_xticks([])
                    ax.set_yticks([])

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


In [ ]:
show_cluster_examples_klscores(output_path, df[df['old_cluster']!=-1], seed=8)